# Training

In [ ]:
!pip install ultralytics --quiet

Conversão das labels

In [ ]:
import os
from PIL import Image

# Caminho para as pastas de imagens e labels
images_path = "/content/drive/MyDrive/Hackaton/Dataset/val/images"
labels_path = "/content/drive/MyDrive/Hackaton/Dataset/val/labels"

# ID da classe (0 para "faca")
class_id = 0

# Função para converter as labels
def convert_labels(image_width, image_height, x_min, y_min, x_max, y_max):
    # Calcular as coordenadas normalizadas
    x_center = ((x_min + x_max) / 2) / image_width
    y_center = ((y_min + y_max) / 2) / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    return f"{class_id} {x_center} {y_center} {width} {height}"

# Percorrer todos os arquivos de labels
for label_file in os.listdir(labels_path):
    if label_file.endswith(".txt"):
        label_file_path = os.path.join(labels_path, label_file)

        # Ler o conteúdo do arquivo
        with open(label_file_path, "r") as file:
            lines = file.readlines()

        # Obter o nome da imagem correspondente
        image_file = label_file.replace(".txt", ".jpg")
        image_path = os.path.join(images_path, image_file)

        # Verificar se a imagem existe
        if not os.path.exists(image_path):
            print(f"Imagem faltando para o label: {image_file}")
            continue

        # Obter as dimensões da imagem
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        # Escrever as labels no formato YOLO
        with open(label_file_path, "w") as file:
            for line in lines:
                # Extrair os valores atuais
                parts = line.strip().split()
                class_name = parts[0]  # "Knife"
                x_min = float(parts[1])  # x_min
                y_min = float(parts[2])  # y_min
                x_max = float(parts[3])  # x_max
                y_max = float(parts[4])  # y_max

                # Converter para o formato YOLO
                yolo_label = convert_labels(image_width, image_height, x_min, y_min, x_max, y_max)

                # Escrever a linha no formato YOLO
                file.write(yolo_label + "\n")

print("Conversão das labels concluída!")

Conversão das labels concluída!


In [ ]:
from ultralytics import YOLO
import os

# 1. Configuração do Dataset
# Certifique-se de que o dataset está organizado da seguinte forma:
# /dataset/
#    ├── train/
#    │   ├── images/
#    │   └── labels/
#    └── val/
#        ├── images/
#        └── labels/

# Caminho para o dataset
dataset_path = "/content/drive/MyDrive/Hackaton/Dataset"

# 2. Criar o arquivo data.yaml
data_yaml = f"""

train: {dataset_path}/train/images
val: {dataset_path}/val/images

# Número de classes
nc: 1

# Nomes das classes
names: ['Knife']
"""

# Salvar o arquivo data.yaml
with open("/content/drive/MyDrive/Hackaton/data.yaml", "w") as file:
    file.write(data_yaml)

# 3. Verificar a estrutura do dataset
def verificar_dataset(dataset_path):
    print("Verificando estrutura do dataset...")
    pastas_necessarias = ["/content/drive/MyDrive/Hackaton/Dataset/train/images", "/content/drive/MyDrive/Hackaton/Dataset/train/labels", "/content/drive/MyDrive/Hackaton/Dataset/val/images", "/content/drive/MyDrive/Hackaton/Dataset/val/labels"]
    for pasta in pastas_necessarias:
        if not os.path.exists(os.path.join(dataset_path, pasta)):
            raise Exception(f"Pasta faltando: {pasta}")
    print("Estrutura do dataset está correta!")

verificar_dataset(dataset_path)

# 4. Carregar um modelo pré-treinado
# YOLOv8n é a versão "nano" (leve e rápida)
model = YOLO("yolo11m.pt")

# 5. Treinar o modelo
print("Iniciando o treinamento...")
results = model.train(
    data="/content/drive/MyDrive/Hackaton/data.yaml",  # Caminho para o arquivo de configuração
    epochs=50,         # Número de épocas
    imgsz=640,         # Tamanho da imagem (640x640 é comum)
    batch=8,           # Tamanho do batch (ajuste conforme sua GPU)
    name="yolov11_knife_detection",  # Nome do experimento
    patience=25,       # Parar o treinamento se não houver melhoria em 10 épocas
    device="cpu",        # Usar GPU (se disponível). Use "cpu" para treinar na CPU.
)

# 6. Avaliar o modelo
print("Avaliando o modelo...")
metrics = model.val()  # Avaliar no conjunto de validação
print(f"mAP50-95: {metrics.box.map}")  # Exibir a métrica mAP

# 7. Testar o modelo em uma imagem de exemplo
print("Testando o modelo em uma imagem de exemplo...")
test_image_path = os.path.join(dataset_path, "/content/drive/MyDrive/Hackaton/Dataset/val/images/096c39dfb17068cf.jpg")  # Substitua pelo caminho de uma imagem de teste
results = model.predict(test_image_path, save=True, conf=0.5)  # conf = limite de confiança

# Exibir os resultados
for result in results:
    result.show()  # Exibir a imagem com as detecções
    result.save()  # Salvar a imagem com as detecções

# 8. Exportar o modelo para ONNX (opcional)
print("Exportando o modelo para ONNX...")
model.export(format="onnx")

# 9. Mensagem final
print("Treinamento concluído!")
print(f"Resultados salvos em: {os.path.abspath('runs/detect/yolov8_knife_detection')}")

# Test
Script para Testar o Modelo em Várias Imagens

In [ ]:
import os
from ultralytics import YOLO


# Carregar o modelo treinado
model = YOLO("/content/drive/MyDrive/Hackaton/yolov11n/best.pt")

# Caminho para as imagens de validação
val_images_path = "/content/drive/MyDrive/Hackaton/Dataset/train/images"

# Testar o modelo em todas as imagens de validação
for image_name in os.listdir(val_images_path):
    if image_name.endswith(".jpg") or image_name.endswith(".png"):
        image_path = os.path.join(val_images_path, image_name)
        print(f"Testando o modelo na imagem: {image_name}")
        results = model.predict(image_path, save=True, conf=0.5)  # conf = limite de confiança
        for result in results:
            result.show()  # Exibir a imagem com as detecções

print("Teste concluído!")

Testar o modelo em um video

In [ ]:
from ultralytics import YOLO

# 1. Carregar o modelo treinado
model = YOLO("/content/drive/MyDrive/Hackaton/yolov11n/best.pt")

# 2. Especificar o caminho do vídeo
video_path = "/content/drive/MyDrive/Hackaton/video.mp4"  # Substitua pelo caminho do seu vídeo

target_classes = [43,76]  # Substitua 47 pelo ID correto da classe faca no seu modelo

# 3. Processar o vídeo
results = model.predict(
    source=video_path,  # Caminho do vídeo
    conf=0.5,           # Limite de confiança para as detecções
    save=True,          # Salvar o vídeo com as detecções
    show=True,           # Exibir o vídeo em tempo real (opcional)
    classes=target_classes  # Filtrar as detecções por classes específicas
)

# 4. Mensagem final
print("Processamento do vídeo concluído!")
print(f"Vídeo com detecções salvo em: {results[0].save_dir}")

# Detection

In [ ]:
!pip install ultralytics --quiet

In [ ]:
import logging
# Configuração do Logger
# logging.basicConfig(filename='app.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

logging.info(f"SYSTEM:: ===== Iniciando programa =====")

INFO:root:SYSTEM:: ===== Iniciando programa =====


In [ ]:
import cv2
import os
from ultralytics import YOLO

# 1. Carregar o modelo treinado
model = YOLO("/content/best.pt")
# model = YOLO("/content/yolo11m.pt")
logging.info(f"DETECTION:: Carregando modelo {model}")


# 2. Especificar o caminho do vídeo
video_path = "/content/video.mp4"

# Caminho para salvar os frames com detecção
output_frames_dir = '/content/frames_detectados'
os.makedirs(output_frames_dir, exist_ok=True)

# 3. Processar o vídeo e contar as detecções
cap = cv2.VideoCapture(video_path)

total_detections = 0
frame_idx = 0
saved_frame_idx = 0

# target_classes = [43, 76]
target_classes = [0]

# Flag para garantir que apenas um frame será salvo por detecção
detected_objects = set()

logging.info(f"DETECTION:: Iniciando processo deteção do video {os.path.basename(video_path)}")

objetos_detectados = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        logging.warning(f'Final do vídeo ou erro de leitura: {video_path}')
        break

    try:
        # Detecção de objetos com YOLO
        results = model(frame)

        for result in results:
            boxes = result.boxes.cpu().numpy()
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                conf = box.conf[0]
                cls = int(box.cls[0])
                # if conf > 0.7:  # Confiança mínima
                label = model.names[cls]
                # logging.warning(f"if1")
                if label in ['Knife', 'scissors', 'machete', 'cutter']:
                    # Desenhar caixa delimitadora
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
                    cv2.putText(frame, f'{label} {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                    frame_filename = os.path.join(output_frames_dir, f"imagem_{objetos_detectados}.jpg")
                    logging.warning(f"DETECTION:: Nova faca encontrada, salvando em {frame_filename}")
                    cv2.imwrite(frame_filename, frame)

                    objetos_detectados += 1
                    logging.info(f'{label} detectado com {conf*100:.1f}% de confiança.')

    except Exception as e:
        logging.error(f'Erro no processamento do frame: {e}')

    # cv2.imshow('VisionGuard - Detecção de Objetos Cortantes (YOLO)', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        logging.info('Análise interrompida pelo usuário.')
        break

cap.release()

In [ ]:
# Mensagem final
# print(f"Processamento do vídeo concluído!")
logging.info(f"DETECTION:: Fim do processo de detecção")
# print(f"Total de detecções: {total_detections}")
logging.info(f"DETECTION:: Total de detecções: {objetos_detectados}")
# print(f"Frames salvos em: {output_frames_dir}")

INFO:root:DETECTION:: Fim do processo de detecção
INFO:root:DETECTION:: Total de detecções: 548


# Mail

In [ ]:
import random

# Definindo variáveis principais
# IMAGENS = ["imagem1", "imagem2"]
IMAGENS_DIR = output_frames_dir
IMAGENS = [f"imagem_{i+1}" for i in range(objetos_detectados)]
random.shuffle(IMAGENS)

QTD = len(IMAGENS)
VIDEO = os.path.basename(video_path)
DESTINATARIOS = ["email1@gmail.com", "email2@gmail.com"]
REMETENTE = "fakevisionguard@gmail.com"

In [ ]:
import smtplib
import ssl

from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.utils import parseaddr
from sys import exit

In [ ]:
# Função para validar email
def validar_email(email):
    nome, endereco = parseaddr(email)
    if '@' in endereco and '.' in endereco.split('@')[-1]:
        return True
    return False

# Função para carregar a senha do arquivo 'secret.txt'
def carregar_senha():
    # try:
    #     with open("secret.txt", "r") as file:
    #         password = file.read().strip()
    #         if not password:
    #             logging.critical("MAIL:: Senha não encontrada")
    #             exit(1)
    #         return password
    # except FileNotFoundError:
    #     logging.critical("MAIL:: O arquivo 'secret.txt' não foi encontrado.")
    #     exit(1)
    return "wixb qyrv qvkc qqwh"

# Função para enviar e-mail
def enviar_email(sender_email, receiver_email, qtd, video, password):
    # Criando parametros da mensagem
    # message = MIMEMultipart("alternative") # Para enviar apenas texto
    message = MIMEMultipart("related") # Para enviar imagens
    message["Subject"] = "Alerta Segurança"
    message["From"] = sender_email
    message["To"] = receiver_email

    # Tratamento de exceção para qtd e video
    if not qtd or not video:
        logging.warning("MAIL:: Informações faltando. QTD ou VIDEO não foi fornecido.")
        qtd = "desconhecido"
        video = "desconhecido"

    # Criando corpo do email com Imagens
    imagens_html = ""
    for i in range(min(len(IMAGENS), 5)):
        imagens_html += f'<img src="cid:{IMAGENS[i]}" alt="Imagem de Alerta" width="250" height="250">'


    # Criando o corpo do email em HTML
    html = f"""
    <html>
        <body style="font-family: Arial, sans-serif; background-color: #f4f4f4; color: #333; padding: 20px;">
            <table role="presentation" style="width: 100%; border: 1px solid #ddd; background-color: #fff; padding: 15px;">
                <tr>
                    <td>
                        <h2 style="color: #d9534f;">Alerta de Segurança</h2>
                        <p>Foram detectados <strong>{qtd}</strong> frames com objetos perigosos durante o escaneamento do vídeo <strong>{video}</strong>.</p>
                        <h3>Amostra de imagens encontradas:</h3>
                        {imagens_html}
                        <p><a href="#" style="color: #007bff; text-decoration: none;">Acesse a analise completa</a></p>
                        <br />
                        <footer style="font-size: 0.9em; color: #777;">
                            <b>Vision Guard Seguradora LTDA.</b><br />
                            Todos os direitos reservados.
                        </footer>
                    </td>
                </tr>
            </table>
        </body>
    </html>
    """

    # Convertendo o HTML para objetos MIMEText e acoplando a mensagem
    body = MIMEText(html, "html")
    message.attach(body)
    logging.info("MAIL:: Corpo do email criado com sucesso")


    for i, img_id in enumerate(IMAGENS[:5]):
        img_path = os.path.join(output_frames_dir, f"{img_id}.jpg")
        with open(img_path, "rb") as img_file:
            img = MIMEImage(img_file.read())
            img.add_header('Content-ID', f'<{img_id}>')
            message.attach(img)
            logging.info(f"MAIL:: Imagem {img_id} adicionada ao e-mail")



    # Criando uma conexão com o servidor e enviando o email
    try:
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            logging.info("MAIL:: Conectando ao servido SMTP")
            server.login(sender_email, password)
            logging.info("MAIL:: Logado no servidor com sucesso")
            server.sendmail(sender_email, receiver_email, message.as_string())
            logging.info(f"MAIL:: Email enviado com sucesso para {receiver_email}")
    except smtplib.SMTPException as e:
        logging.error(f"MAIL:: Erro ao enviar o email: {str(e)}")
    except Exception as e:
        logging.critical(f"MAIL:: Erro inesperado: {str(e)}")

In [ ]:
# Validando os emails
if not validar_email(REMETENTE):
    print(f"Endereço de e-mail do remetente inválido: {REMETENTE}")
    exit(1)

DESTINATARIOS_VALIDOS = []
for email in DESTINATARIOS:
    if validar_email(email):
        DESTINATARIOS_VALIDOS.append(email)
    else:
        print(f"Endereço de e-mail inválido: {email}")

if not DESTINATARIOS_VALIDOS:
    print("Nenhum destinatário válido encontrado.")
    exit(1)

# Carregando a senha do arquivo 'secret.txt'
PASSWORD = carregar_senha()

# Enviando os e-mails
sender_email = REMETENTE
for receiver_email in DESTINATARIOS_VALIDOS:
    enviar_email(sender_email, receiver_email, QTD, VIDEO, PASSWORD)

logging.info(f"SYSTEM:: ===== Finalizando programa =====")